<a href="https://colab.research.google.com/github/drod75/Weeaboo-Buddy/blob/main/notebooks/Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Package installation

In [1]:
!pip install -U langchain-community tavily-python python-dotenv langchain-google-genai langgraph langchain-tavily

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_google_genai import ChatGoogleGenerativeAI

## Testing

In [3]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [11]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [13]:
from langchain_community.tools.tavily_search import TavilySearchResults
from google.colab import userdata

import os

tkey = userdata.get('TAVILY_API_KEY')
if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = tkey

gkey = userdata.get('GOOGLE_API_KEY')
if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = gkey

tool = TavilySearchResults(max_results=4)


In [14]:

query = "Who won the super bowl in 2024? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question."
messages = [HumanMessage(content=query)]

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
abot = Agent(model, [tool], system='You are an American football expert who does research of every team and there stats each season.')
result = abot.graph.invoke({"messages": messages})

print(result['messages'][-1].content)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'who won super bowl 2024'}, 'id': '6d9d8b1b-cf1d-4d9c-851a-e34241f6d9ae', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'GDP of Missouri'}, 'id': 'e6889dbf-909d-407b-b697-4fe12c81b5da', 'type': 'tool_call'}
Back to the model!
The Kansas City Chiefs won the Super Bowl in 2024. The team's headquarters are located in Missouri. The GDP of Missouri in 2023 was approximately $348.49 billion.


## Testing 2

In [16]:
prompt = '''
You are a crunchyroll data analyst who specialzes in researching various different types of anime to help questions for fans,
as well as other analysts who specialize in drawing up graph and statistics.
'''
query = 'Who is Zoro from One Piece'

messages = [HumanMessage(content=query)]

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

print(result['messages'][-1].content)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Who is Zoro from One Piece'}, 'id': '16651fc6-6462-4bd7-9ed5-9fdbc88fc1e7', 'type': 'tool_call'}
Back to the model!
Roronoa Zoro, also known as "Pirate Hunter" Zoro, is the combatant of the Straw Hat Pirates. He is a master swordsman who uses a unique Three Sword Style technique. He was the first member to join the Straw Hat Pirates and is considered one of the top fighters in the crew, recognized alongside Luffy and Sanji as one of the "Monster Trio". Zoro's ambition is to become the world's strongest swordsman.


In [17]:
! pip install jikanpy-v4

In [18]:
from jikanpy import Jikan
jikan = Jikan()

bebop = jikan.anime(1)
bebop

{'data': {'mal_id': 1,
  'url': 'https://myanimelist.net/anime/1/Cowboy_Bebop',
  'images': {'jpg': {'image_url': 'https://cdn.myanimelist.net/images/anime/4/19644.jpg',
    'small_image_url': 'https://cdn.myanimelist.net/images/anime/4/19644t.jpg',
    'large_image_url': 'https://cdn.myanimelist.net/images/anime/4/19644l.jpg'},
   'webp': {'image_url': 'https://cdn.myanimelist.net/images/anime/4/19644.webp',
    'small_image_url': 'https://cdn.myanimelist.net/images/anime/4/19644t.webp',
    'large_image_url': 'https://cdn.myanimelist.net/images/anime/4/19644l.webp'}},
  'trailer': {'youtube_id': 'gY5nDXOtv_o',
   'url': 'https://www.youtube.com/watch?v=gY5nDXOtv_o',
   'embed_url': 'https://www.youtube.com/embed/gY5nDXOtv_o?enablejsapi=1&wmode=opaque&autoplay=1',
   'images': {'image_url': 'https://img.youtube.com/vi/gY5nDXOtv_o/default.jpg',
    'small_image_url': 'https://img.youtube.com/vi/gY5nDXOtv_o/sddefault.jpg',
    'medium_image_url': 'https://img.youtube.com/vi/gY5nDXOtv_o/

In [24]:
current_season = jikan.seasons(extension='now')
current_season

{'pagination': {'last_visible_page': 9,
  'has_next_page': True,
  'current_page': 1,
  'items': {'count': 25, 'total': 210, 'per_page': 25}},
 'data': [{'mal_id': 51818,
   'url': 'https://myanimelist.net/anime/51818/Enen_no_Shouboutai__San_no_Shou',
   'images': {'jpg': {'image_url': 'https://cdn.myanimelist.net/images/anime/1527/146836.jpg',
     'small_image_url': 'https://cdn.myanimelist.net/images/anime/1527/146836t.jpg',
     'large_image_url': 'https://cdn.myanimelist.net/images/anime/1527/146836l.jpg'},
    'webp': {'image_url': 'https://cdn.myanimelist.net/images/anime/1527/146836.webp',
     'small_image_url': 'https://cdn.myanimelist.net/images/anime/1527/146836t.webp',
     'large_image_url': 'https://cdn.myanimelist.net/images/anime/1527/146836l.webp'}},
   'trailer': {'youtube_id': 'nz-VCl7yUAw',
    'url': 'https://www.youtube.com/watch?v=nz-VCl7yUAw',
    'embed_url': 'https://www.youtube.com/embed/nz-VCl7yUAw?enablejsapi=1&wmode=opaque&autoplay=1',
    'images': {'ima

In [23]:
search_result = jikan.search('anime', 'Boku no Piko')
search_result

{'pagination': {'last_visible_page': 5,
  'has_next_page': True,
  'current_page': 1,
  'items': {'count': 25, 'total': 125, 'per_page': 25}},
 'data': [{'mal_id': 1639,
   'url': 'https://myanimelist.net/anime/1639/Boku_no_Pico',
   'images': {'jpg': {'image_url': 'https://cdn.myanimelist.net/images/anime/12/39497.jpg',
     'small_image_url': 'https://cdn.myanimelist.net/images/anime/12/39497t.jpg',
     'large_image_url': 'https://cdn.myanimelist.net/images/anime/12/39497l.jpg'},
    'webp': {'image_url': 'https://cdn.myanimelist.net/images/anime/12/39497.webp',
     'small_image_url': 'https://cdn.myanimelist.net/images/anime/12/39497t.webp',
     'large_image_url': 'https://cdn.myanimelist.net/images/anime/12/39497l.webp'}},
   'trailer': {'youtube_id': None,
    'url': None,
    'embed_url': None,
    'images': {'image_url': None,
     'small_image_url': None,
     'medium_image_url': None,
     'large_image_url': None,
     'maximum_image_url': None}},
   'approved': True,
   't

In [35]:
from langchain_core.tools import tool

@tool
def search_anime(query: str) -> dict:
    """
    Name:
      search_anime
    Description:
      Search for an anime by a name, the result will be a json of possile solutions, from best to worst match.
    Args:
      query: anime name
    """

    search_result = jikan.search('anime', query)
    return search_result


@tool
def get_stats_by_id(id: int) -> dict:
    """
    Name:
      get_stats_by_id
    Description:
      Get the stats of an anime by it's id
    Args:
      id: anime id
    """

    result = jikan.anime(id)
    return result

@tool
def get_seasonal() -> dict:
    """
    Name:
      get_seasonal
    Description:
      Get the anime from this season
    Args:
      None
    """
    current_season = jikan.seasons(extension='now')
    return current_season

tavily = TavilySearchResults(max_results=4)
tools = [search_anime, get_stats_by_id, get_seasonal, tavily]


In [36]:
prompt = '''
You are a crunchyroll data analyst who specialzes in researching various different types of anime to help questions for fans,
as well as other analysts who specialize in drawing up graph and statistics.
'''
query = 'What are the anime from this current season?'

messages = [HumanMessage(content=query)]

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
abot = Agent(model, tools, system=prompt)
result = abot.graph.invoke({"messages": messages})

print(result['messages'][-1].content)

Calling: {'name': 'get_seasonal', 'args': {}, 'id': 'd543db90-7727-4469-b590-4ad322167c03', 'type': 'tool_call'}
Back to the model!
Okay, I have the anime from the current season. Here are the first 25 titles:

1.  Enen no Shouboutai: San no Shou (Fire Force Season 3)
2.  Lazarus
3.  Wind Breaker Season 2
4.  Witch Watch
5.  Saikyou no Ousama, Nidome no Jinsei wa Nani wo Suru? (The Beginning After the End)
6.  Tu Bian Yingxiong X (To Be Hero X)
7.  Vigilante: Boku no Hero Academia Illegals (My Hero Academia: Vigilantes)
8.  Kowloon Generic Romance
9.  Katainaka no Ossan, Kensei ni Naru (From Old Country Bumpkin to Master Swordsman)
10. Slime Taoshite 300-nen, Shiranai Uchi ni Level Max ni Nattemashita: Sono Ni (I've Been Killing Slimes for 300 Years and Maxed Out my Level Season 2)
11. Danjo no Yuujou wa Seiritsu suru? (Iya, Shinai!!) (Can a Boy-Girl Friendship Survive?)
12. Haite Kudasai, Takamine-san (Please Put Them On, Takamine-san)
13. Kijin Gentoushou (Sword of the Demon Hunter: 